In [1]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.indices import SummaryIndex
from pathlib import Path
import chromadb
from llama_index.vector_stores import ChromaVectorStore
# load papers


papers_category = ["Covid", 'GNN', 'InBody']
paper_vector_index = {}
paper_summary_index = {}
for paper in papers_category:
    
    # storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{paper.key}_vector")
    # paper_vector_index[paper.key] = load_index_from_storage(storage_context)

    # storage_context = StorageContext.from_defaults(persist_dir=f"./storage/cache/{paper.key}_summary")
    # paper_summary_index[paper.key] = load_index_from_storage(storage_context)
    
    try:
        # # load index
        #     pass
        db = chromadb.PersistentClient(path=f'./storage/chroma/{paper}_index')
        chroma_collection =db.get_or_create_collection(f"{paper}")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store,
                                                    persist_dir=f'./storage/chroma/{paper}_index')
        paper_vector_index[paper] = load_index_from_storage(storage_context)
        # VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
        # vector_index.storage_context.persist(persist_dir=f"./storage/cache/{paper.key}_vector")

        
        db = chromadb.PersistentClient(path=f'./storage/chroma/{paper}_summary')
        chroma_collection =db.get_or_create_collection(f"{paper}")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store,
                                                    persist_dir=f'./storage/chroma/{paper}_summary')
        paper_summary_index[paper] = load_index_from_storage(storage_context)
        # VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
        # paper_summary_index[paper].storage_context.persist(persist_dir=f"./storage/cache/{paper.key}_summary")
        
        # new_index = load_index_from_storage(storage_context)
        print('load index')
        
    except:
    # make index
    
        loader = SimpleDirectoryReader(input_dir=Path(f"./assets/Papers/{paper}"))
        documents = loader.load_data()
        
        db = chromadb.PersistentClient(path=f'./storage/chroma/{paper}_index')
        chroma_collection =db.get_or_create_collection(f"{paper}")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        paper_vector_index[paper] = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
        paper_vector_index[paper].storage_context.persist(persist_dir=f"./storage/chroma/{paper}_index")

        
        db = chromadb.PersistentClient(path=f'./storage/chroma/{paper}_summary')
        chroma_collection =db.get_or_create_collection(f"{paper}")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)
        paper_summary_index[paper] = SummaryIndex.from_documents(documents, storage_context=storage_context)
        paper_summary_index[paper].storage_context.persist(persist_dir=f"./storage/chroma/{paper}_summary")
        print('make index')



OperationalError: attempt to write a readonly database

In [3]:
paper_vector_index, paper_summary_index

({'Covid': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x17b680940>,
  'GNN': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x1035ee6e0>,
  'InBody': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x2896ffdf0>},
 {'Covid': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x17b681240>,
  'GNN': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x289858b50>,
  'InBody': <llama_index.indices.vector_store.base.VectorStoreIndex at 0x2894bebf0>})

In [ ]:
from llama_index.llms import OpenAI

In [5]:
from llama_index.chat_engine import ContextChatEngine
from llama_index.retrievers import (
    VectorIndexRetriever,
    VectorIndexAutoRetriever,
    SummaryIndexRetriever,
    SummaryIndexEmbeddingRetriever,
    SummaryIndexLLMRetriever,
)
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.core.llms.types import ChatMessage, MessageRole
from llama_index.postprocessor.types import BaseNodePostprocessor
from typing import Any, List, Optional, Tuple
    
paper_summary_engines = {}
query_engine_tools = []
for paper in papers_category:
    index_retriever = VectorIndexRetriever(
        index = paper_vector_index[paper],
        similarity_top_k = 3,
    )
    
    summary_index_retriever = SummaryIndexRetriever(
        index = paper_summary_index[paper],
        similarity_top_k = 3,
    )
    # summary_embed, summary_llm은 너무 느림 -> 사용불가능
    # summary_embed_retriever = SummaryIndexEmbeddingRetriever(
    #     index = paper_summary_index[paper],
    #     similarity_top_k = 3
    # )
    # summary_llm_retriever = SummaryIndexLLMRetriever(
    #     index = paper_summary_index[paper],
    #     similarity_top_k = 3,
    # )
    
    
    
    # # VectorIndexAutoRetriever : 메타데이터 정보를 포함하여 검색하는 리트리버
    # index_auto_retriever = VectorIndexAutoRetriever(
    #     index = paper_vector_index[paper],
    #     similarity_top_k = 3,
    # )
    # break
    
    # chat_engine = index_retriever.as_chat_engine(chat_mode="react")
    
    chat_engine = ContextChatEngine.from_defaults(
            retriever = index_retriever,
            # prefix_messages = List[ChatMessage],
            # node_postprocessors = Optional[List[BaseNodePostprocessor]]
        )
    
    query_engine_tool = QueryEngineTool(
        query_engine=chat_engine,
        metadata=ToolMetadata(
            name=paper,
            description=f"Provides information about the cat breed {paper}"
        ),
    )
    
    query_engine_tools.append(query_engine_tool)

# DEFAULT_CONTEXT_TEMPLATE = (
#     "Context information is below."
#     "\n--------------------\n"
#     "{context_str}"
#     "\n--------------------\n"
# )
subquestion_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [6]:
res1 = index_retriever.retrieve('What is RLHF?')

In [7]:
res2 = summary_index_retriever.retrieve('What is RLHF?')

AttributeError: 'IndexDict' object has no attribute 'nodes'

In [ ]:
# Chat Engine 

In [28]:
res1.__len__(), res2.__len__(), res3.__len__(), res4.__len__()

(3, 106, 3, 25)

In [34]:
for i in range(3):
    print(res1[i].metadata)#, res1[i].get_content())
    print(res2[i].metadata)#, res2[i].get_content())
    print('---------------------------------')

{'page_label': '5', 'file_name': 'Predicting the Dynamics of the COVID-19 Pandemic in the United States Using Graph Theory-Based Neural Networks.pdf', 'file_path': 'assets/Papers/Covid/Predicting the Dynamics of the COVID-19 Pandemic in the United States Using Graph Theory-Based Neural Networks.pdf', 'file_type': 'application/pdf', 'file_size': 39542939, 'creation_date': '2024-02-18', 'last_modified_date': '2022-07-11', 'last_accessed_date': '2024-02-18'}
{'page_label': '1', 'file_name': 'Dynamic graph based epidemiological model for COVID-19 contact tracing data analysis and optimal testing prescription.pdf', 'file_path': 'assets/Papers/Covid/Dynamic graph based epidemiological model for COVID-19 contact tracing data analysis and optimal testing prescription.pdf', 'file_type': 'application/pdf', 'file_size': 719384, 'creation_date': '2024-02-18', 'last_modified_date': '2022-08-11', 'last_accessed_date': '2024-02-18'}
{'page_label': '5', 'file_name': 'Predicting the Dynamics of the COV

In [13]:
subquestion_engine.query('what is covid?')

Generated 1 sub questions.
[Covid] Q: What is Covid?


AttributeError: 'ContextChatEngine' object has no attribute 'aquery'

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata # 3개이기 때문에 쿼리엔진에 대한 설명이 필요함

query_engine_tools = []
for breed in breeds:
    query_engine = vector_qurey_engines[breed]
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=breed,
            description=f"Provides information about the cat breed {breed}"
        ),
    )
    query_engine_tools.append(query_engine_tool)

subquestion_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
from llama_index.indices.postprocessor import KeywordNodePostprocessor
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.tools import QueryEngineTool, ToolMetadata

node_processor = KeywordNodePostprocessor(
    exclude_keywords=["commecial"]
)

agents = {}
query_engine_tools = []
paper_vector_engines = {}
paper_summary_engines = {}

for paper in papers_category:
    retriever = VectorIndexRetriever(
        index = paper_vector_index[paper],
        similarity_top_k = 10,
    )
    paper_vector_engines[paper] = RetrieverQueryEngine(
        retriever = retriever,
        node_postprocessors = [node_processor]
    )
    
    paper_summary_engines[paper] = paper_summary_index[paper].as_query_engine()

    
    new_tool = QueryEngineTool(
        query_engine=paper_vector_engines[paper],
        metadata=ToolMetadata(
            name=f"{paper}_vector_tool",
            description=f"Useful for retrieving specific context from a paper {paper}. "
            f"Use When you need information related to {paper}"
            )
    )
    
    query_engine_tools.append(new_tool)
    
    new_tool = QueryEngineTool(
        query_engine=paper_summary_engines[paper],
        metadata=ToolMetadata(
            name=f"{paper}_summaey_tool",
            description=f"Useful for summary specific context from a paper {paper}. "
            f"Use When you need overview information related to {paper}"
            )
    )
    
    query_engine_tools.append(new_tool)
    
    from llama_index.agent import OpenAIAgent
    from llama_index.llms import OpenAI

    agent = OpenAIAgent.from_tools(query_engine_tools, llm=OpenAI(temperature=0, model='gpt-3.5-turbo-0613'), verbose=True)
    agents[paper] = agent

In [ ]:
# create index
from llama_index import VectorStoreIndex, SimpleDirectoryReader

llama_2_docs = SimpleDirectoryReader(input_files=['./assets/llama2.pdf'], filename_as_id=True).load_data()
index = VectorStoreIndex.from_documents(llama_2_docs)
chat_engine = index.as_chat_engine(chat_mode="context")




In [ ]:
# create chat engine
from llama_index.chat_engine import ChatEngine

response = chat_engine.chat("What is Llama2?")